## _*Chemistry with Qiskit Aqua and t|ket〉*_  
### Excited states with Quantum Subspace Expansion

The energies of electronic excited states prove, in general, challenging to compute. Excited states are usually more entangled than ground states and so require more computational resources to compute. This causes the number of states considered in a calculation to grow considerably, often beyond the capabilities of the most advanced classical supercomputers. This is a problem as it makes classical computation of the energy, to an appropriate accuracy, of many molecules impossible. Quantum computers are inherently capable of handling calculations with entangled states and can provide a workable solution.

This notebook demonstrates calculation of excited states of molecules using quantum algorithms. This has recently been achieved by IBM with efficient circuit designs using exchange-type gates.[1] 

However this example shows Qiskit Aqua interface combined with fast circuit compilation and optimization via `t|ket〉`.  It makes use of the Quantum Subspace Expansion [2, 3] as implemented in `pytket` (the python interface to `t|ket〉`). To install it go to http://github.com/CQCL/pytket and follow the README (or just try running `pip install pytket`).

In order to perform this calculation, we start from a ground state wavefunction $\Psi_0$ calculated with the Variational Quantum Eigensolver (VQE) technique. We construct a subspace of state vectors $\Psi_i^k$ formed by one-electron excitations of the ground state wavefunction:

\begin{equation}
\left|\Psi_i^k\right\rangle = c_k^{\dagger}c_i\left|\Psi_0\right\rangle
\end{equation}

where $c_k^{\dagger}$, $c_i$ are the fermionic creation and annihilation operators over spinorbitals k and i, respectively. The fermionic operators act by reducing the occupation of spinorbital i to zero, and increasing the occupation of spinorbital k to one. Then we solve the generalized eigenvalue problem corresponding to this subspace, whose equation is:

\begin{equation}
HC=SCE
\end{equation}

where $C$ is the matrix of eigenvectors, $E$ is the vector of eigenvalues, $S$ is the overlap matrix and $H$ is the Hamiltonian matrix for the subspace considered. The elements h$_{ij}^{kl}$ of $H$ are of the form:

\begin{equation}
h_{ij}^{kl} =  \langle\Psi_i^k \left| \widehat{H} \right| \Psi_j^l\rangle
\end{equation}

Where $\widehat{H}$ is the molecular hamiltonian operator. In a similar way, the elements s$_{ij}^{kl}$ of $S$ are found from the following expression:

\begin{equation}
s_{ij}^{kl} = \langle \Psi_i^k \left|\Psi_j^l\right\rangle
\end{equation}

The matrix elements can be calculated using a quantum computer or simluator.

To perform the evaluations corresponding to each element, we can use the following trick. In the second quantization formalism, S matrix element $s_{ij}^{kl}$ the element may be written as:

\begin{equation}
 s_{ij}^{kl} = \langle ik  | jl \rangle= \langle \Psi_0 c_i c_k^\dagger |c_l^\dagger c_j \Psi_0 \rangle =\langle \Psi_0 |c_i c_k^\dagger c_l^\dagger c_j|\Psi_0 \rangle
\end{equation}

We can obtain $s^{ij}_{kl}$ by transforming the $c_ic_k^\dagger c_l^\dagger c_j$ set of operators to a set of Pauli quantum gates according to an appropriate scheme such as Jordan-Wigner or Bravyi-Kitaev, apply this gate set to the ground state wavefunction (constructed with the coefficients obtained from the VQE calculation) and perform a measurement to obtain the expected value. In a similar fashion, the H matrix elements can be measured after constructing the operator $c_ic_k^\dagger \hat{H}c_l^\dagger c_j$, transforming it to a representation of Pauli operators and applying it over $|\Psi_0\rangle$. 

Let’s illustrate this method with a simple example, the lithium hydride (LiH) molecule. We first run the VQE procedure using the tket〉 compiler and Qiskit simulator to determine the LiH ground state energy and construct the H and S matrix elements required for the QSE algorithm. First let's start by importing qiskit and pytket.

In [1]:
import qiskit
import pytket

## Configure experiment <a id='sectionB'></a>

We choose the basis set spanning the molecular wavefunction, the molecular geometry, the chemical identity of each atom, the charge and spin quantum number.

To get results for LiH (slower), just comment out the H2 string and replace it with the LiH one.

In [2]:
bond_length = 0.7

# setup molecule
# base_molecule_str = 'Li .0 .0 .0; H .0 .0 {}'
base_molecule_str = 'H .0 .0 .0; H .0 .0 {}'
charge = 0
spin = 0
basis = 'sto3g'

We now choose a backend to run the quantum sub-routines on. Running the experiment on a real device requires setting up an account first.
Note: If you do not store your token yet, use IBMQ.save_accounts() to store it first.

In [3]:
# choose quantum backend (e.g. 'statevector_simulator', 'qasm_simulator', 'ibmqx5')
# for a remote backend you will need to load your API account

# local backends
backend_name = 'statevector_simulator'
backend = qiskit.Aer.get_backend(backend_name)

# remote backends
# qiskit.IBMQ.load_accounts()
# backend_name = 'ibmqx5'
# backend = qiskit.IBMQ.get_backend(backend)

## t|ket〉compilation <a id='sectionC'></a>

We now set up a `QuantumInstance` object with our backend and specify that we want to use the `TketPass` transpilation pass from `pytket` as part of our compilation of quantum circuits. The pass performs chemistry specific optimisation and significantly reduces circuit depth for these routines. The table below exhibits the difference in circuit size and depth for ground state energy estimation of LiH via VQE using Qiskit and `t|ket〉`. Routing refers to the process of making quantum circuits hardware compliant by the addition of SWAP gates such that all multi-qubit interactions occur on adjacent physical qubits. The following routing results are for circuits compliant to Qiskit's ibmqx5 computer.

![alt text](LiHCompilation.png "Title")

In [4]:
from qiskit_aqua import QuantumInstance
from qiskit.transpiler import PassManager
from pytket.qiskit import TketPass

pass_manager = PassManager()
tk_pass = TketPass(backend)
pass_manager.append(tk_pass)

quantum_instance = QuantumInstance(backend, pass_manager=pass_manager)

## Perform classical calculation <a id='sectionB'></a>

We execute our classical chemistry driver to obtain the hamiltonian integrals. In this case, we choose PYSCF as our driver, so make sure you have that installed.


In [5]:
from qiskit_aqua_chemistry.drivers import ConfigurationManager

# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
driver = cfg_mgr.get_driver_instance('PYSCF')

pyscf_cfg = {'atom': base_molecule_str.format(bond_length),
             'unit': 'Angstrom',
             'charge': charge,
             'spin': spin,
             'basis': basis}

molecule = driver.run({'properties': pyscf_cfg})

print("Molecular repulsion energy: ", molecule.nuclear_repulsion_energy)

Molecular repulsion energy:  0.7559674441714287


## Set up initial state and variational form <a id='sectionB'></a>

We define the molecular hamiltonian and map it to a qubit operator according to some appropriate scheme such as the Jordan-Wigner transform. We initialize our initial state (a Hartree-Fock state) and our variational ansatz (a unitary coupled cluster with single and double excitations, UCCSD).

In [6]:
from qiskit_aqua_chemistry import FermionicOperator
from qiskit_aqua_chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit_aqua_chemistry.aqua_extensions.components.variational_forms import UCCSD

n_qubits = molecule.one_body_integrals.shape[0]
n_electrons = molecule.num_alpha + molecule.num_beta - molecule.molecular_charge

# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.chop(10**-10)

# setup variation form generator (generate trial circuits for VQE)
initial_hf = HartreeFock(num_qubits=n_qubits, num_orbitals=n_qubits, 
                    qubit_mapping='jordan_wigner', two_qubit_reduction=False, num_particles= n_electrons)

var_form = UCCSD(num_qubits=n_qubits, num_orbitals=n_qubits, 
                num_particles=n_electrons, depth=1, initial_state=initial_hf, qubit_mapping='jordan_wigner')





## Use VQE to find ground state <a id='sectionB'></a>

We run the VQE algorithm with the components previously defined. The calculation provides us with the ground state energy and optimized parameters for the corresponding UCCSD wavefunction. In this example we use the L_BFGS_B algorithm for classical optimization.

In [ ]:
from qiskit_aqua.components.optimizers import L_BFGS_B
from qiskit_aqua.algorithms import VQE


# set initial values of parameters
number_amplitudes = len(var_form._single_excitations)+ len(var_form._double_excitations)

amplitudes_0 = []
for i in range(number_amplitudes):
    amplitudes_0.append(0.00001)

optimizer = L_BFGS_B()
optimizer.set_options(maxfun=1000, factr=10, iprint=10)

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = VQE(operator=qubitOp, operator_mode='matrix', 
                    var_form=var_form, optimizer=optimizer, initial_point=amplitudes_0)

results = vqe_algorithm.run(quantum_instance)

eigval = results['eigvals'][0]
gs_energy = eigval.real + molecule.nuclear_repulsion_energy

print("GS Minimum value: {}".format(gs_energy))
print("GS Parameters: {}".format(results['opt_params']))

# store ground state amplitudes for subsequent steps
opti_amplitudes = results['opt_params']



## Use QSE to find excited states from ground state <a id='sectionB'></a>

We now have the main ingredients to perform a QSE calculation: the molecular hamiltonian and the optimized parameters to reconstruct the ground state wavefunction. We build our excitation hamiltonian and overlap operators, and measure the elements that compose $H$ and $S$. After we have obtained these arrays, we perform a diagonalization to obtain the excited state energies and vectors.


In [ ]:
from pytket.chemistry import QseMatrices, QSE

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
n_qubits = qubitOp.num_qubits
qubitOp.chop(10**-10)
# use matrix term helper class
matrix_terms = QseMatrices(qubitOp, n_qubits)

qse_algorithm = QSE(matrix_terms, 'matrix', var_form, opt_init_point=opti_amplitudes)

energies = qse_algorithm.run(quantum_instance)['eigvals']
print("Excited State Energies: ", energies+molecule.nuclear_repulsion_energy)



The calculation provides a refined value of the ground state and a series of excited state energies, whose number depends on the size of the basis set chosen for the molecule, as well as its nature and symmetry. Some of the energies obtained are repeated several times, signaling that some of the states obtained are degenerate. This result can be improved by either increasing the basis set or considering higher-order excitations in the subspace expansion.

The following graph shows us the excited states of LiH at a range of bond distances calculated via two methods. Using our quantum algorithm, we find the ground state curve has a minimum at a separation of about 1.5 Å, which is in reasonable agreement with experimental data. The calculation also finds a number of excited states. Looking at the first three of these, we find that at the equilibrium distance, these states are 0.11, 0.12 and 0.17 Ha higher in energy than the ground state, which is again in reasonable agreement with experimental data. Note the small kink in one of the excited states energy curve at a distance of approximately 1.2 Å. This indicates that our restriction to single electron excitations is not enough to provide an accurate description at this distance. Overall, the comparison with classically computed EOM-CCSD curves shows that this method reproduces excited state energies with good accuracy at most distances.

![alt text](LiH.png "Title")

[1] Ganzhorn. M , Egger. D. J. Barkoutsos.P, Ollitrault. P, Salis. G, Moll. N, Fuhrer. A, Mueller. P, Woerner. S, Tavernelli. I, Filipp. (IBM) S arXiv: 1809.05057v1

[2] J. R. McClean, M. E. Kimchi-Schwartz, J. Carter and W. A. de Jong, Phys. Rev. A 95, 042308 (2017)

[3] J. I. Colless, V. V. Ramasesh, D. Dahlen, M. S. Blok, M. E. Kimchi-Schwartz, J. R. McClean, J. Carter, W. A. de Jong and I. Siddiqi, Phys. Rev. X 8, 011021 (2018)

